In [3]:
import cv2
import numpy as np

def filtering(img, filt):
    height = img.shape[0] 
    width = img.shape[1]  
    row = height - filt.shape[0] + 1 
    col = width - filt.shape[1] + 1  
    result = np.zeros((height, width), np.uint8)
    
    for r in range(0, row) :
        for c in range(0, col):
            total = np.sum(img[r:r+filt.shape[0], c:c+filt.shape[1]] * filt)
            result[r+1,c+1] = total
    
    return result

def zero_crossing(img):
    h = img.shape[0]                          ## LoG filter 취한 이미지의 행과 열의 개수
    w = img.shape[1]
    thres = np.absolute(img).mean() * 0.75    ## threshold 값은 전체 pixel 절대갑의 평균 * 0.75
    output = np.zeros([h,w])                  ## binary를 찍을 output
    
    for y in range(1, h - 1):             
        for x in range(1, w - 1):
            patch = img[y-1:y+2, x-1:x+2]     ## 3x3 pixel (영교차를 검출하기 위해서 설정)
            p = img[y, x]                     ## 중심 pixel
            maxP = patch.max()                ## 3x3 pixel의 최대값
            minP = patch.min()                ## 3x3 pixel의 최소값
            
            if (p > 0):                                     ## 중심 pixel이 밝은데  주변 pixel이 어둡거나
                zeroCross = True if minP < 0 else False   ##  중심 pixel이 어두운데, 주변 pixel이 밝으면
            else:                                           ## 영교차가 있다고 간주
                zeroCross = True if maxP > 0 else False
            if ((maxP - minP) > thres) and zeroCross:       ## 3x3 pixel의 최대값과 최소값의 차가 threshold를 넘으면 Edge 추출
                output[y, x] = 1
    return output


img = cv2.imread("opencv/picture/newyork.jpg", cv2.IMREAD_GRAYSCALE)                 ## 이미지 그레이스케일 변환
img = cv2.resize(img,(640,480))                                                      ## 이미지 해상도 변경
          
x, y = np.meshgrid(np.linspace(-300,300,3), np.linspace(-300,300,3))                 ## 600개 sampling, 3x3행렬 생성
d = np.sqrt(x*x+y*y)
sigma, mu = 15.0, 0.0                                         ## 표준편차 값이 커질수록 명암대비 밝은 곳에 Edge가 선명해진다.
g_filter = np.exp(-( (d-mu)**2 / ( 2.0 * (sigma**2) ) ) ) / np.sqrt(2*np.pi)*sigma  ## Gaussian Filter 생성
g_filter = g_filter / np.sum(g_filter)

log_filter = ((y**2 + x**2 - 2*(sigma**2)) / (sigma**4)) * g_filter                 ## LoG Filter 생성

img_log = np.array(filtering(img, log_filter))                                        ## Convolution 수행

result = zero_crossing(img_log)                                                       ## 영교차 탐색

cv2.imshow('original', img)
cv2.imshow('result', result)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [212]:
if(p > 0):      ## 0쪽으로 갈 수록 어둡고 255로 갈 수록 밝다. p>0이냐는건 3x3행렬의 중앙이 밝은 pixel인지 물어보는듯.
    if minP<0:  ## 근처 8개 pixel중에 0보다 작은값이 있는 놈은 어둡다
        zeroCross = True  ## 따라서 밝은 pixel p의 주변에 0보다 작은 값이 있다는것은 어두운값에서 밝은 값으로 확 변하는 영교차가 있다는 뜻일거임.
    else:                 ## 그게 아니라면 영교차는 존재하지 않는다.
        zeroCross = False

False